<a href="https://colab.research.google.com/github/13194307/UTS_ML2019_ID13194307/blob/master/ML_A2/NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO LIST:


*   Finish off backpropagation (currently only updates output layer weights)
*   Perhaps add different kinds of layer classes (one for Dense layers, one for output)



In [0]:
import math
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from scipy.special import softmax
from scipy import stats

In [0]:
class NeuralNetwork:
    class Layer:
        class Neuron:
            def __init__(self, inputShape):
                self.weights, self.bias = self.initialiseWeights(inputShape)
                
            def initialiseWeights(self, inputShape):
                weights = np.array([np.random.randn() for _ in range(0, inputShape)]) / math.sqrt(inputShape)
                bias = np.random.randn()
                
                return weights, bias
            
            def getWeightsAndBias(self):
                return self.weights, self.bias
            
            def generateNeuronOutput(self, x):
                return np.dot(self.weights, x) + self.bias
            
            def updateWeights(self, update):
                print("Before:", self.weights)
                self.weights -= update
                print("After:", self.weights)
            
        
        def __init__(self, layerType, neuronsPerLayer, inputShape):
            self.layerType = layerType
            self.numNeurons = neuronsPerLayer
            self.inputShape = inputShape
            self.neurons = [self.Neuron(inputShape) for _ in range(neuronsPerLayer)]
            self.dWeightedSum = []
            
            if layerType == "Output":
                self.dSoftmax = []
            
        def getInputShape(self):
            return self.inputShape
        
        def getNumNeurons(self):
            return self.numNeurons
        
        def getDSoftmax(self):
            temp = self.dSoftmax
            self.dSoftmax = []
            return temp
        
        def getDWeightedSum(self):
            temp = self.dWeightedSum
            self.dWeightedSum = []
            return temp
        
        def generateLayerOutput(self, x):
            layerOutput = np.array([])
            change = [self.calcDWeightedSum(x) for _ in range(self.numNeurons)]
            self.dWeightedSum.append(change)
            
            for neuron in self.neurons:
                neuronOutput = neuron.generateNeuronOutput(x)
                layerOutput = np.append(layerOutput, neuronOutput)
               
            print(self.layerType, ":", layerOutput)
            if self.layerType == "Dense":
                #Leaky ReLU activation
                layerOutput[layerOutput < 0] *= 0.01
            elif self.layerType == "Output":
                if len(layerOutput) == 1:
                    #Sigmoid activation
                    layerOutput = 1 / (1 + math.exp(-1*layerOutput[0]))
                    #print("Probabilities: ", layerOutput)
                else:
                    #Softmax activation
                    layerOutput = np.exp(layerOutput)/sum(np.exp(layerOutput))
                    change = self.calcDSoftmax(layerOutput)
                    self.dSoftmax.append(change)
                    #print(self.dSoftmax)
                    #print("Probabilities: ", layerOutput)
            else:
                raise NotImplementedError
                    
            print("After activation - ", layerOutput)
            return layerOutput
        
        # Derivative of softmax with respect to weighted sum/dot product
        def calcDSoftmax(self, prob):
            n = len(prob)
            return [prob[i]-(prob[i]**2) for i in range(n)]
        
        # Derivative of weighted sum with respect to the inputs provided
        # This function is kinda pointless as it just returns the argument
        # passed to it unaltered but I added it in to remind me that this
        # is the derivative.
        def calcDWeightedSum(self, x):
            return x
        
        def updateWeights(self, update):
            for i in range(self.numNeurons):
                self.neurons[i].updateWeights(update[i])
            
        def __str__(self):
            output = ""
            for neuron in self.neurons:
                weights, bias = neuron.getWeightsAndBias()
                output+="\t"
                
                for j in range(0, len(weights)):
                    output+=("w{}: {}, ".format(j, weights[j]))
                    
                output+=("b0: {}\n".format(bias))
            
            return output
        
        
        
        
        
    
    def __init__(self):
        self.layers = []
        self.numLayers = 0
        
    def addLayer(self, layerType, neuronsPerLayer, inputShape=None):
        if inputShape == None:
            inputShape = self.layers[-1].getNumNeurons()
            
        self.layers.append(self.Layer(layerType, neuronsPerLayer, inputShape))
        self.numLayers+=1
        
    def predict(self, x, labels, batch_size=2, step_size = 0.1):
        probabilities = []
        dError = []
        counter = 0
        
        for i in range(len(x)):
            prob = self.feedForward(x[i])
            change = self.calcDError(prob, labels[i])
            dError.append(change)
            counter+=1
            
            if counter == batch_size:
                counter = 0
                self.backPropagation(dError, step_size, batch_size)
                dError = []
                
            #print(pred)
            probabilities.append(prob)
      
        predictions = np.argmax(probabilities, axis=1)
        #loss = self.calcTotalLoss(probabilities, labels)
        #print(loss)
        return predictions
    
    def feedForward(self, x):
        lastLayerOutput = x
        
        for layer in self.layers:
            lastLayerOutput = layer.generateLayerOutput(lastLayerOutput)
            
        return lastLayerOutput
    
    def backPropagation(self, dError, step_size, batch_size=None):
        #Only updates output layer weights for now
        d1 = np.array(dError)
        d2 = np.array(self.layers[-1].getDSoftmax())
        d3 = np.array(self.layers[-1].getDWeightedSum())
        print("d1:", d1)
        print("d2:", d2)
        print("d3:", d3[0])
        changeOutputWeights = [d1[i]*d2[i]*d3[i] for i in range(len(d3))]
        avgChange = (sum(changeOutputWeights)/batch_size) * step_size
        print("change:", changeOutputWeights)
        print("fin change:", avgChange)
        self.layers[-1].updateWeights(avgChange)
    
    #Derivative of error (cross-entropy) with respect to softmax probabilities
    def calcDError(self, prob, actual):
        n = len(actual)
        return [-1*(actual[i]/prob[i]) + ((1 - actual[i])*(-1/(1-prob[i]))) for i in range(n)]
    
    def calcTotalLoss(self, prob, actual):
        #REDO THIS TO WORK WITH ONE HOT ENCODING
        n = len(actual)
        loss = -1*sum(np.log([prob[i][actual[i]] for i in range(n)]))/n
        return loss
        
    def __str__(self):
        output = ""
        
        for i in range(0, self.numLayers):
            output+=("Layer {}:\n".format(i+1))
            output+=str(self.layers[i])
        
        return output

In [0]:
nn = NeuralNetwork()
nn.addLayer("Dense", 3, inputShape=4)
nn.addLayer("Dense", 3)
nn.addLayer("Dense", 3)
nn.addLayer("Output", 3)
#print(nn)

In [0]:
from sklearn.datasets import load_iris

iris_X, iris_y = load_iris(True)

In [0]:
from sklearn.preprocessing import MinMaxScaler

iris_X_trimmed = iris_X

scaler = MinMaxScaler()
iris_X_scaled = scaler.fit_transform(iris_X_trimmed)
iris_X_zscore = stats.zscore(iris_X_trimmed)

In [0]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
labels = lb.fit_transform(iris_y)

In [223]:
pred = nn.predict(iris_X_zscore[0:2, :], labels[0:2])
accuracy_score(pred, iris_y[0:2])

Dense : [ 1.43230935 -0.58692942 -0.96182649]
After activation -  [ 1.43230935 -0.00586929 -0.00961826]
Dense : [ 1.00770779 -1.20973112  0.86316142]
After activation -  [ 1.00770779 -0.01209731  0.86316142]
Dense : [-2.34949951  0.68492261  1.04320793]
After activation -  [-0.023495    0.68492261  1.04320793]
Output : [-0.29776819 -1.45048303  0.90796942]
After activation -  [0.21482279 0.06783638 0.71734083]
Dense : [ 0.97796934 -1.45725378 -0.21187656]
After activation -  [ 0.97796934 -0.01457254 -0.00211877]
Dense : [ 0.93231188 -0.91179989  0.35670832]
After activation -  [ 0.93231188 -0.009118    0.35670832]
Dense : [-1.85887643  0.33822026  1.08473955]
After activation -  [-0.01858876  0.33822026  1.08473955]
Output : [-0.15729058 -1.80649352  1.03280554]
After activation -  [0.22323421 0.04290629 0.7338595 ]
d1: [[-4.65499952 -1.07277304 -3.53782964]
 [-4.47960024 -1.04482977 -3.75741385]]
d2: [[0.16867396 0.06323461 0.20276296]
 [0.17340069 0.04106534 0.19530973]]
d3: [[-0.023

0.0

In [67]:
print(iris_y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
